In [1]:
import os
from knowledge_storm import STORMWikiRunnerArguments, STORMWikiRunner, STORMWikiLMConfigs
from knowledge_storm.lm import OpenAIModel
from knowledge_storm.rm import YouRM

In [2]:
lm_configs = STORMWikiLMConfigs()
openai_kwargs = {
    'api_key': os.getenv("OPENAI_API_KEY"),
    'temperature': 1.0,
    'top_p': 0.9,
}
# STORM is a LM system so different components can be powered by different models to reach a good balance between cost and quality.
# For a good practice, choose a cheaper/faster model for `conv_simulator_lm` which is used to split queries, synthesize answers in the conversation.
# Choose a more powerful model for `article_gen_lm` to generate verifiable text with citations.
gpt_35 = OpenAIModel(model='gpt-3.5-turbo', max_tokens=500, **openai_kwargs)
gpt_4 = OpenAIModel(model='gpt-4o', max_tokens=3000, **openai_kwargs)
lm_configs.set_conv_simulator_lm(gpt_4)
lm_configs.set_question_asker_lm(gpt_4)
lm_configs.set_outline_gen_lm(gpt_4)
lm_configs.set_article_gen_lm(gpt_4)
lm_configs.set_article_polish_lm(gpt_4)


In [3]:
lm_configs.log()

OrderedDict([('conv_simulator_lm',
              {'temperature': 1.0,
               'max_tokens': 3000,
               'top_p': 0.9,
               'frequency_penalty': 0,
               'presence_penalty': 0,
               'n': 1,
               'model': 'gpt-4o'}),
             ('question_asker_lm',
              {'temperature': 1.0,
               'max_tokens': 3000,
               'top_p': 0.9,
               'frequency_penalty': 0,
               'presence_penalty': 0,
               'n': 1,
               'model': 'gpt-4o'}),
             ('outline_gen_lm',
              {'temperature': 1.0,
               'max_tokens': 3000,
               'top_p': 0.9,
               'frequency_penalty': 0,
               'presence_penalty': 0,
               'n': 1,
               'model': 'gpt-4o'}),
             ('article_gen_lm',
              {'temperature': 1.0,
               'max_tokens': 3000,
               'top_p': 0.9,
               'frequency_penalty': 0,
               'presenc

In [4]:
# Check out the STORMWikiRunnerArguments class for more configurations.
engine_args = STORMWikiRunnerArguments("outputs")
rm = YouRM(ydc_api_key=os.getenv('YDC_API_KEY'), k=engine_args.search_top_k)
runner = STORMWikiRunner(engine_args, lm_configs, rm)

In [5]:
topic = "Llama 3 (Language Model)"

In [6]:
runner.run(
    topic=topic,
    do_research=True,
    do_generate_outline=True,
    do_generate_article=True,
    do_polish_article=True,
)
runner.post_run()
runner.summary()

knowledge_storm.interface : INFO     : run_knowledge_curation_module executed in 133.1669 seconds
knowledge_storm.interface : INFO     : run_outline_generation_module executed in 7.1936 seconds
sentence_transformers.SentenceTransformer : INFO     : Use pytorch device_name: mps
sentence_transformers.SentenceTransformer : INFO     : Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
knowledge_storm.interface : INFO     : run_article_generation_module executed in 16.1945 seconds
knowledge_storm.interface : INFO     : run_article_polishing_module executed in 237.9508 seconds


***** Execution time *****
run_knowledge_curation_module: 133.1669 seconds
run_outline_generation_module: 7.1936 seconds
run_article_generation_module: 16.1945 seconds
run_article_polishing_module: 237.9508 seconds
***** Token usage of language models: *****
run_knowledge_curation_module
    gpt-4o: {'prompt_tokens': 29876, 'completion_tokens': 18213}
run_outline_generation_module
    gpt-4o: {'prompt_tokens': 489, 'completion_tokens': 407}
run_article_generation_module
    gpt-4o: {'prompt_tokens': 11345, 'completion_tokens': 3271}
run_article_polishing_module
    gpt-4o: {'prompt_tokens': 3184, 'completion_tokens': 475}
***** Number of queries of retrieval models: *****
run_knowledge_curation_module: {'YouRM': 36}
run_outline_generation_module: {'YouRM': 0}
run_article_generation_module: {'YouRM': 0}
run_article_polishing_module: {'YouRM': 0}
